# Predicting whether & how a Red Devil will play in the next match 

### importing packages

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib as plt
import seaborn as sns

### importing dataset

In [2]:
data = pd.read_csv('Dataset_ManUtd_Player_Performance_2019-20.csv')

In [3]:
data.fillna(0,inplace=True)
data.head()

,Last Name,First Name,Age,Appearances this EPL Season,Appearances in last 5 matches,Goals Scored this EPL Season,Goals Assisted this EPL Season,Own Goals this EPL Season,Goals Scored in last 5 matches,Goals Assited in last 5 matches,...,Yellow Cards in last 5 matches,Red Cards in last 5 matches,Penalty Goals this EPL Season,Penalty Goals in last 5 matches,Minutes Per Goal this EPL Season,Total Minutes Played this EPL Season,Total minutes in last 5 matches,Injury Status this EPL Season,Form this EPL Season,Form in last 5 matches
0,Rashford,Marcus,22,12,5.0,6.0,3.0,0.0,5.0,0.0,...,1.0,0.0,2.0,0.0,173.0,1040.0,417.0,0.00,6.2,0.0
1,Wan Bissaka,Aaron,21,10,5.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,891.0,441.0,0.00,1.8,0.0
2,McTominay,Scott,22,12,5.0,3.0,1.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,356.0,1067.0,111.0,0.75,4.2,0.0
3,Pogba,Paul,26,5,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,450.0,0.0,1.00,0.0,0.0
4,Martial,Anthony,23,7,5.0,3.0,3.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,177.0,531.0,369.0,0.00,4.8,0.0


In [4]:
data.describe()

,Age,Appearances this EPL Season,Appearances in last 5 matches,Goals Scored this EPL Season,Goals Assisted this EPL Season,Own Goals this EPL Season,Goals Scored in last 5 matches,Goals Assited in last 5 matches,Own Goals in last 5 matches,Goals Conceded this EPL Season,...,Yellow Cards in last 5 matches,Red Cards in last 5 matches,Penalty Goals this EPL Season,Penalty Goals in last 5 matches,Minutes Per Goal this EPL Season,Total Minutes Played this EPL Season,Total minutes in last 5 matches,Injury Status this EPL Season,Form this EPL Season,Form in last 5 matches
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.0,28.000000,28.000000,28.0,28.000000,...,28.000000,28.0,28.000000,28.0,28.000000,28.000000,28.000000,28.000000,28.000000,28.0
mean,24.500000,5.821429,2.214286,0.642857,0.464286,0.0,0.392857,0.250000,0.0,0.428571,...,0.285714,0.0,0.071429,0.0,65.392857,420.750000,160.178571,0.223214,1.232143,0.0
std,4.434712,4.538343,2.233701,1.445665,0.922241,0.0,1.065947,0.700529,0.0,2.267787,...,0.712697,0.0,0.377964,0.0,173.317593,417.258568,173.838378,0.380906,1.791968,0.0
min,18.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,21.750000,1.750000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,20.250000,0.000000,0.000000,0.000000,0.0
50%,24.500000,5.000000,1.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,258.500000,100.500000,0.000000,0.000000,0.0
75%,27.000000,10.250000,5.000000,0.000000,0.250000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,820.500000,295.000000,0.500000,1.850000,0.0
max,34.000000,12.000000,5.000000,6.000000,3.000000,0.0,5.000000,3.000000,0.0,12.000000,...,3.000000,0.0,2.000000,0.0,797.000000,1080.000000,450.000000,1.000000,6.200000,0.0
